# SuperGlue Alignment

In [ ]:
import numpy as np
import cv2
import os
import matplotlib.pyplot as plt
import jupyter_compare_view

In [ ]:
# https://stackoverflow.com/questions/7099290/how-to-ignore-hidden-files-using-os-listdir
def listdir_nohidden(path):
    for f in os.listdir(path):
        if not f.startswith('.'):
            yield f

In [ ]:
# Feel free to edit as you like depending on where you put the files
BASE_IMAGE_DIR = "VIS_IR_DATASET"
SAVE_DIR = "ALIGNED_DATASET"

os.makedirs(SAVE_DIR, exist_ok=True)

In [ ]:
fileNames = listdir_nohidden(BASE_IMAGE_DIR)
filePairs = []
for fileName in fileNames:
    im = cv2.imread(f"{BASE_IMAGE_DIR}/{fileName}")
    
    # For simplicity only keep the file name, not extension
    fileName = fileName.split('.')[0]
    
    # Rescale
    rescale_percent = 0.5
    width = int(im.shape[1] * rescale_percent)
    height = int(im.shape[0] * rescale_percent)
    dim = (width, height)
    im = cv2.resize(im, dim, interpolation = cv2.INTER_AREA)
    
    # Split the side by side image into two halfs
    im_ir = im[:, (im.shape[1]//2):, :]
    im_vis = im[:, :(im.shape[1]//2), :]
    
    cv2.imwrite(f"{SAVE_DIR}/{fileName}_ir.jpg", im_ir)
    cv2.imwrite(f"{SAVE_DIR}/{fileName}_vis.jpg", im_vis)
    
    filePairs.append([f"{fileName}_ir.jpg", f"{fileName}_vis.jpg"]) # Subject, Template

# After done writing the individual images, write pairs.txt for SuperGlue
with open(f"{SAVE_DIR}/pairs.txt", "w") as f:
    for [vis, ir] in filePairs:
        f.write(f"{vis} {ir}\n")

# Run the SuperGlue match finder here: 
Using a command like this: ```./SuperGluePretrainedNetwork/match_pairs.py --resize 1640 1232 --superglue outdoor --max_keypoints 2048 --nms_radius 3  --resize_float --input_dir ALIGNED_DATASET --input_pairs ALIGNED_DATASET/pairs.txt  --output_dir ALIGNED_DATASET/superglue --viz```

# Using the SuperGlue output to align images

In [ ]:
def combine_vis_ir(im_vis, im_ir, H, w, h, save_dir, file_name):
    im_aligned = cv2.warpPerspective(im_ir, H, (w, h))
    im_aligned_grey = cv2.cvtColor(im_aligned, cv2.COLOR_BGR2GRAY).reshape((im_vis.shape[0], im_vis.shape[1], 1))
    im_combined = np.concatenate((im_vis, im_aligned_grey), axis=2)
    os.makedirs(save_dir, exist_ok=True)
    np.save(f"{save_dir}/{file_name}", im_combined)

In [ ]:
fileNames = listdir_nohidden(BASE_IMAGE_DIR)
excludeList = ["2021-11-15_22-59-29", "2021-11-15_22-59-51", "2021-11-15_22-59-54", "2021-11-15_22-56-31"]
filePairs = []

for fileName in fileNames:
    fileName = fileName.split(".")[0]
    
    if fileName in excludeList:
        continue
    
    im_vis = cv2.imread(f"{SAVE_DIR}/{fileName}_vis.jpg")
    im_ir = cv2.imread(f"{SAVE_DIR}/{fileName}_ir.jpg")
    
    npz = np.load(f"{SAVE_DIR}/superglue/{fileName}_ir_{fileName}_vis_matches.npz")
    
    best_idx = npz['match_confidence'] > 0.85
    best_kpt0 = npz['keypoints0'][best_idx]
    best_kpt1 = []
    for i, idx in enumerate(best_idx):
        if idx:
            assert npz['matches'][i] != -1, "Index was not found in match"
            best_kpt1.append(npz['keypoints1'][npz['matches'][i]])
    best_kpt1 = np.array(best_kpt1)
    
    assert best_kpt0.shape == best_kpt1.shape, \
        f"File: {fileName} failed, shape {best_kpt0.shape} is different from shape {best_kpt1.shape}"
    
    assert best_kpt0.shape[0] >= 4, \
        f"File: {fileName} failed, does not have at least 4 points. It has {best_kpt0.shape[0]} instead."
    
    (H, mask) = cv2.findHomography(best_kpt0, best_kpt1, method=cv2.RANSAC)
    (h, w) = im_vis.shape[:2]
    
    combine_vis_ir(im_vis, im_ir, H, w, h, f"{SAVE_DIR}/stacked/", f"{fileName}_combined.npy")

In [ ]:
%%compare
fig, ax1 = plt.subplots()
ax1.axis("off")
ax1.imshow(cv2.cvtColor(im_vis, cv2.COLOR_BGR2RGB))

# Align and convert the 
fig, ax2 = plt.subplots()
ax2.axis("off")
aligned = cv2.warpPerspective(im_ir, H, (w, h))
ax2.imshow(cv2.cvtColor(aligned, cv2.COLOR_BGR2RGB))